In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
from load_cifar import *
import time

In [2]:
tf.reset_default_graph()
validation_data, validation_labels = load_preprocessed_validation_batch()

<h1>Define Hyper-perparmeter</h1>

In [9]:
lr = 0.0001
num_epochs = 10
batch_size = 128
n_hidden_1 = 500
n_hidden_2 = 300
n_hidden_3 = 200
num_input = 3072
num_classes = 10

<h1>Define Placeholder</h1>

In [10]:
# tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

<h1>Define Neural Network Architecture</h1>

In [11]:
layer_1 = tf.layers.dense(X, n_hidden_1, activation=tf.nn.relu)
# layer_1_bn = tf.layers.batch_normalization(layer_1)
layer_2 = tf.layers.dense(layer_1, n_hidden_2, activation=tf.nn.relu)
# layer_2_bn = tf.layers.batch_normalization(layer_2)
layer_3 = tf.layers.dense(layer_2, n_hidden_3, activation=tf.nn.relu)
# layer_3_bn = tf.layers.batch_normalization(layer_3)
logits = tf.layers.dense(layer_3, num_classes, activation=None)

<h1>Define cost andoptimization</h1>

In [12]:
# define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

# compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(logits),1),tf.argmax(Y,1))

# compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        s = time.time()
        batch_id = 1
        while batch_id < 6:
            training_data = load_preprocessed_training_batch(batch_id, batch_size)
            for data, labels in training_data:
                sess.run(train_op, feed_dict = {X: data, Y: labels})
            batch_id = batch_id + 1
        val_acc = sess.run(accuracy, feed_dict={X: validation_data, Y: validation_labels})
        f = time.time()
        print('Epoch:' + str(i) + '   Time:{:.3f}'.format(f - s) + '   Current validation accuracy: ' + str(val_acc))
    test_data, test_labels = load_all_test_batch()
    test_acc = sess.run(accuracy, feed_dict = {X: test_data, Y: test_labels})
    print('Test accuracy: ')
    print(test_acc)

Epoch:0   Time:2.944   Current validation accuracy: 0.3822
Epoch:1   Time:2.638   Current validation accuracy: 0.4168
Epoch:2   Time:2.635   Current validation accuracy: 0.438
Epoch:3   Time:2.634   Current validation accuracy: 0.4522
Epoch:4   Time:2.632   Current validation accuracy: 0.4654
Epoch:5   Time:2.637   Current validation accuracy: 0.4722
Epoch:6   Time:2.637   Current validation accuracy: 0.4844
Epoch:7   Time:2.625   Current validation accuracy: 0.4848
Epoch:8   Time:2.622   Current validation accuracy: 0.4912
Epoch:9   Time:2.623   Current validation accuracy: 0.5002
Test accuracy: 
0.5049


I've used several kinds of fully connected neural network with different number of neurals in each layer. I found that there is always a limitation of a certain kind of network. For example, when I use two layers with 500 plus 300 neurals, the validation accuracy will stuck at about 0.47 unless increasing the number of epoch. I used 500 plus 300 neurals in homework 1 and the effect isn't bad. At first, I revised my network based more on the validation accuracy, for one reason is that accuracy, to some extent, could tell whether the network performed better or not, for another reason is that I didn't implement the test part by that time. I add one more layer with 100 neruals first and it couldn't break the bound, so I increase the number of neurals to 200. Activation function ReLu has a more better effect compared with common sigmoid and tanh, I used it for all three layers. As for the batch size, I modified it with the learning rate. I found that a big batch size needed a smaller learning rate. I've tried 32, 64, 128 batch size and 128 performs better, so I decide to adjust the learning rate with the 128 batch size. For the initialization, regularization, dropout and normalization part, they are somehow to avoid the overfitting situation. For my network here, I implement normalization which has tiny influences. I think the whole architecture of network is more important. The cost and optimization part is just like those in homework 1. Like the result shown above, I also try 20 epoch and the validation rate fluctuated around 0.52 which I think it may the limitation of this network.